In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv('nba_game_details_cleaned.csv')

In [3]:
df['YEAR'] = pd.to_datetime(df['GAME_DATE']).dt.year

In [4]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

In [5]:
df = df.sort_values(by='GAME_DATE')

In [6]:
df.head()

,GAME_ID,GAME_DATE,TEAM_away,TEAM_home,TEAM_ID_away,TEAM_ID_home,FGM_away,FGM_home,FGA_away,FGA_home,...,BLK_away,BLK_home,TOV_away,TOV_home,PF_away,PF_home,PTS_away,PTS_home,WIN_home,YEAR
1228,21200003,2012-10-30,DAL,LAL,1610612742,1610612747,40,38,85,77,...,5,5,12,14,25,21,99,91,0,2012
1227,21200001,2012-10-30,WAS,CLE,1610612764,1610612739,32,36,90,79,...,10,5,13,21,19,21,84,94,1,2012
1226,21200002,2012-10-30,BOS,MIA,1610612738,1610612748,39,43,75,79,...,2,5,16,8,23,20,107,120,1,2012
1220,21200012,2012-10-31,LAL,POR,1610612747,1610612757,36,42,72,83,...,2,5,25,12,29,20,106,116,1,2012
1225,21200005,2012-10-31,DEN,PHI,1610612743,1610612755,33,30,88,85,...,5,11,22,16,22,14,75,84,1,2012


In [ ]:
df = df.set_index('GAME_DATE')

In [ ]:
df

In [ ]:
df.columns

In [7]:
# Create calculated columns for three-point attempts and makes
df['3PA_away'] = (df['PTS_away'] - df['FTM_away'] - 2*(df['FGM_away']))/(df['FG3_PCT_away'])
df['3PA_away'] = df['3PA_away'].round(0)

df['3PA_home'] = (df['PTS_home'] - df['FTM_home'] - 2*(df['FGM_home']))/(df['FG3_PCT_home'])
df['3PA_home'] = df['3PA_home'].round(0)

df['3PM_away'] = df['PTS_away'] - df['FTM_away'] - 2*df['FGM_away']
df['3PM_home'] = df['PTS_home'] - df['FTM_home'] - 2*df['FGM_home']
                                                      

In [ ]:
df['3P_ratio_away'] = df['3PA_away'] / df['FGA_away']

df['3P_ratio_home'] = df['3PA_home'] / df['FGA_home']

In [8]:
# Merge Hornets/Pelicans history
df['TEAM_home'] = np.where(df['TEAM_home'] == 'NOH', 'NOP', df['TEAM_home'])

In [ ]:
df.head()

In [9]:
threes_df = df.groupby(['TEAM_home', 'YEAR'], as_index=False).mean()
# threes_df['WIN_away'] = threes_df(1 - threes_df['WIN_home'])
threes_df_plotly = threes_df[['TEAM_home', 'YEAR', 'FG3_PCT_home', 'WIN_home', '3PA_home']]
# threes_df_plotly.reset_index(drop=True, inplace=True)
# threes_df_plotly.index.name = "TEAM_YEAR"

In [10]:
js = threes_df_plotly.to_json('threes.json')
js

In [16]:
### Bubble Chart
fig = px.scatter(threes_df.query(f'YEAR=={2021}'), x='3PA_home', y='WIN_home',
                 color='FG3_PCT_home', hover_name='TEAM_home', text="TEAM_home", 
                 labels={
                     "3PA_home": "Three-Point Attempts Per Game",
                     "WIN_home": "Win Rate",
                     "FG3_PCT_home": "Three-Point Pct%"
                 }, 
                 title="Win Percentage vs Three Point Attempts by Team (2021)")
fig.update_traces(textposition='top center')

fig.show()

In [ ]:
# Timeline of 3pa average per year
threes_avg_timeline = df[['3PA_home','3PA_away']].resample('Y').mean()
fig = px.line(threes_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average 3PA'},title='Average 3PA for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013 ,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013, 2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
# Timeline of 3pt ratio average per year
ratio_avg_timeline = df[['3P_ratio_home','3P_ratio_away']].resample('Y').mean()
fig = px.line(ratio_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average 3PA'},title='Average 3PA for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013 ,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013, 2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
# Timeline of AST average per year
ast_avg_timeline = df[['AST_home','AST_away']].resample('Y').mean()
fig = px.line(ast_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average AST'},title='Average AST for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013 ,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013, 2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
# Timeline of FGA average per year
fga_avg_timeline = df[['FGA_home','FGA_away']].resample('Y').mean()
fig = px.line(fga_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average FGA'},title='Average FGA for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
# Timeline of REB average per year
reb_avg_timeline = df[['REB_home','REB_away']].resample('Y').mean()
fig = px.line(reb_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average REB'},title='Average REB for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
# Timeline of FGA average per year
fg3_avg_timeline = df[['FG3_PCT_home','FG3_PCT_away']].resample('Y').mean()
fig = px.line(fg3_avg_timeline, labels={'GAME_DATE':'Year', 'value':'Average FG3_PCT'},title='Average FG3_PCT for home/away by year')
fig.update_layout(xaxis = dict(
        tickvals = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022],
        ticktext = [2013,2014, 2015, 2016, 2017, 2018, 2019,2020,2021,2022]
))
fig.show()

In [ ]:
threes = pd.read_csv('nba_threes.csv')

In [ ]:
threes_js = threes.to_json('nba_threes.json')